# Tusker Loyalty Customer Segmentation Analysis
## Achieving Silhouette Score >0.85 for Customer Clustering

This notebook demonstrates customer segmentation using unsupervised learning techniques with a focus on achieving high clustering performance (Silhouette score >0.85) as specified in the PRD.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.model_selection import GridSearchCV

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

In [ ]:
# Load the training data
train_df = pd.read_csv('../data/train_data.csv')
test_df = pd.read_csv('../data/test_data.csv')

print(f"Training data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")
print(f"\nTraining data columns: {list(train_df.columns)}")

# Display basic info
train_df.head()

In [ ]:
# Exploratory Data Analysis
def create_eda_visualizations(df):
    """Create comprehensive EDA visualizations"""
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Tusker Loyalty Customer Data - Exploratory Analysis', fontsize=16, fontweight='bold')
    
    # Age distribution
    axes[0,0].hist(df['age'], bins=30, alpha=0.7, color='#DAA520')
    axes[0,0].set_title('Age Distribution')
    axes[0,0].set_xlabel('Age')
    axes[0,0].set_ylabel('Frequency')
    
    # Income distribution
    income_counts = df['income_kes'].value_counts()
    axes[0,1].pie(income_counts.values, labels=income_counts.index, autopct='%1.1f%%')
    axes[0,1].set_title('Income Distribution')
    
    # Purchase history vs engagement
    axes[0,2].scatter(df['purchase_history'], df['engagement_rate'], alpha=0.6, color='#228B22')
    axes[0,2].set_title('Purchase History vs Engagement Rate')
    axes[0,2].set_xlabel('Purchase History')
    axes[0,2].set_ylabel('Engagement Rate')
    
    # County distribution (top 10)
    top_counties = df['county'].value_counts().head(10)
    axes[1,0].barh(range(len(top_counties)), top_counties.values)
    axes[1,0].set_yticks(range(len(top_counties)))
    axes[1,0].set_yticklabels(top_counties.index)
    axes[1,0].set_title('Top 10 Counties')
    axes[1,0].set_xlabel('Customer Count')
    
    # Upgrade engagement distribution
    axes[1,1].hist(df['upgrade_engagement'], bins=30, alpha=0.7, color='#00BFFF')
    axes[1,1].set_title('M-Pesa Upgrade Engagement (2025)')
    axes[1,1].set_xlabel('Upgrade Engagement Score')
    axes[1,1].set_ylabel('Frequency')
    
    # Segment target distribution
    segment_counts = df['segment_target'].value_counts()
    axes[1,2].bar(range(len(segment_counts)), segment_counts.values)
    axes[1,2].set_xticks(range(len(segment_counts)))
    axes[1,2].set_xticklabels(segment_counts.index, rotation=45, ha='right')
    axes[1,2].set_title('Target Segments')
    axes[1,2].set_ylabel('Count')
    
    plt.tight_layout()
    plt.show()

create_eda_visualizations(train_df)

In [ ]:
# Feature Engineering and Preprocessing
def preprocess_features(df):
    """Preprocess features for clustering"""
    
    df_processed = df.copy()
    
    # Encode categorical variables
    le_income = LabelEncoder()
    le_behavior = LabelEncoder()
    le_county = LabelEncoder()
    le_profit = LabelEncoder()
    
    df_processed['income_encoded'] = le_income.fit_transform(df_processed['income_kes'])
    df_processed['behavior_encoded'] = le_behavior.fit_transform(df_processed['behavior'])
    df_processed['county_encoded'] = le_county.fit_transform(df_processed['county'])
    df_processed['profit_encoded'] = le_profit.fit_transform(df_processed['profit_segment'])
    
    # Select features for clustering
    feature_cols = [
        'age', 'income_encoded', 'purchase_history', 'behavior_encoded',
        'engagement_rate', 'county_encoded', 'profit_encoded', 'upgrade_engagement'
    ]
    
    X = df_processed[feature_cols]
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    return X_scaled, feature_cols, (le_income, le_behavior, le_county, le_profit, scaler)

# Preprocess training data
X_train_scaled, feature_cols, encoders = preprocess_features(train_df)
print(f"Processed features shape: {X_train_scaled.shape}")
print(f"Feature columns: {feature_cols}")

In [ ]:
# Optimize K-means clustering for Silhouette score >0.85
def find_optimal_clusters(X, max_k=15):
    """Find optimal number of clusters using multiple metrics"""
    
    silhouette_scores = []
    inertias = []
    k_range = range(2, max_k + 1)
    
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        cluster_labels = kmeans.fit_predict(X)
        
        silhouette_avg = silhouette_score(X, cluster_labels)
        silhouette_scores.append(silhouette_avg)
        inertias.append(kmeans.inertia_)
        
        print(f"k={k}: Silhouette Score = {silhouette_avg:.4f}")
    
    # Plot results
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Silhouette scores
    ax1.plot(k_range, silhouette_scores, 'bo-', linewidth=2, markersize=8)
    ax1.axhline(y=0.85, color='r', linestyle='--', label='Target Score (0.85)')
    ax1.set_xlabel('Number of Clusters (k)')
    ax1.set_ylabel('Silhouette Score')
    ax1.set_title('Silhouette Score vs Number of Clusters')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Elbow method
    ax2.plot(k_range, inertias, 'ro-', linewidth=2, markersize=8)
    ax2.set_xlabel('Number of Clusters (k)')
    ax2.set_ylabel('Inertia')
    ax2.set_title('Elbow Method')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Find best k for target silhouette score
    best_scores = [(k, score) for k, score in zip(k_range, silhouette_scores) if score > 0.85]
    
    if best_scores:
        best_k = min(best_scores, key=lambda x: x[0])[0]  # Choose smallest k that meets criteria
        print(f"\n✅ Found k={best_k} with Silhouette score > 0.85")
    else:
        best_k = k_range[np.argmax(silhouette_scores)]
        print(f"\n⚠️ Best k={best_k} with Silhouette score = {max(silhouette_scores):.4f} (below 0.85)")
    
    return best_k, silhouette_scores

# Find optimal clusters
optimal_k, scores = find_optimal_clusters(X_train_scaled)

In [ ]:
# Advanced clustering with PCA for better performance
def optimize_clustering_with_pca(X, target_score=0.85):
    """Use PCA to improve clustering performance"""
    
    # Try different PCA components
    best_score = 0
    best_config = None
    
    for n_components in [3, 4, 5, 6]:
        pca = PCA(n_components=n_components, random_state=42)
        X_pca = pca.fit_transform(X)
        
        print(f"\nPCA with {n_components} components (explained variance: {pca.explained_variance_ratio_.sum():.3f})")
        
        for k in range(2, 12):
            kmeans = KMeans(n_clusters=k, random_state=42, n_init=20)
            labels = kmeans.fit_predict(X_pca)
            score = silhouette_score(X_pca, labels)
            
            if score > best_score:
                best_score = score
                best_config = (n_components, k, pca, kmeans, X_pca)
            
            if score > target_score:
                print(f"  k={k}: Silhouette = {score:.4f} ✅")
            else:
                print(f"  k={k}: Silhouette = {score:.4f}")
    
    print(f"\n🎯 Best configuration: PCA({best_config[0]}) + K-means({best_config[1]}) = {best_score:.4f}")
    
    return best_config

# Optimize with PCA
best_config = optimize_clustering_with_pca(X_train_scaled)
n_components, best_k, pca, best_kmeans, X_pca = best_config

In [ ]:
# Final model training and evaluation
final_labels = best_kmeans.predict(X_pca)
final_score = silhouette_score(X_pca, final_labels)

print(f"🏆 FINAL MODEL PERFORMANCE")
print(f"Silhouette Score: {final_score:.4f}")
print(f"Target Achievement: {'✅ PASSED' if final_score > 0.85 else '❌ NEEDS IMPROVEMENT'}")
print(f"Number of Clusters: {best_k}")
print(f"PCA Components: {n_components}")

# Add cluster labels to training data
train_df_clustered = train_df.copy()
train_df_clustered['predicted_cluster'] = final_labels

# Cluster analysis
print(f"\n📊 CLUSTER DISTRIBUTION:")
cluster_counts = pd.Series(final_labels).value_counts().sort_index()
for cluster, count in cluster_counts.items():
    print(f"Cluster {cluster}: {count:,} customers ({count/len(final_labels):.1%})")

In [ ]:
# Visualize clusters
def visualize_clusters(X_pca, labels, title="Customer Segments - PCA Visualization"):
    """Create 2D and 3D cluster visualizations"""
    
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('2D PCA View', '3D PCA View'),
        specs=[[{'type': 'scatter'}, {'type': 'scatter3d'}]]
    )
    
    # 2D visualization
    colors = px.colors.qualitative.Set3[:len(np.unique(labels))]
    
    for i, cluster in enumerate(np.unique(labels)):
        mask = labels == cluster
        fig.add_trace(
            go.Scatter(
                x=X_pca[mask, 0],
                y=X_pca[mask, 1],
                mode='markers',
                name=f'Cluster {cluster}',
                marker=dict(color=colors[i], size=4, opacity=0.7)
            ),
            row=1, col=1
        )
    
    # 3D visualization
    if X_pca.shape[1] >= 3:
        for i, cluster in enumerate(np.unique(labels)):
            mask = labels == cluster
            fig.add_trace(
                go.Scatter3d(
                    x=X_pca[mask, 0],
                    y=X_pca[mask, 1],
                    z=X_pca[mask, 2],
                    mode='markers',
                    name=f'Cluster {cluster}',
                    marker=dict(color=colors[i], size=3, opacity=0.6),
                    showlegend=False
                ),
                row=1, col=2
            )
    
    fig.update_layout(
        title=title,
        height=600,
        showlegend=True
    )
    
    fig.show()

# Create visualizations
visualize_clusters(X_pca, final_labels)

In [ ]:
# Save the trained model and preprocessing pipeline
import joblib
import os

model_dir = '../models'
os.makedirs(model_dir, exist_ok=True)

# Save components
joblib.dump(pca, f'{model_dir}/pca_transformer.pkl')
joblib.dump(best_kmeans, f'{model_dir}/kmeans_model.pkl')
joblib.dump(encoders, f'{model_dir}/encoders.pkl')

# Save model metadata
model_info = {
    'silhouette_score': final_score,
    'n_clusters': best_k,
    'n_pca_components': n_components,
    'feature_columns': feature_cols,
    'target_achieved': final_score > 0.85
}

import json
with open(f'{model_dir}/model_info.json', 'w') as f:
    json.dump(model_info, f, indent=2)

print("✅ Model and preprocessing pipeline saved successfully!")
print(f"📁 Saved to: {model_dir}")
print(f"🎯 Final Silhouette Score: {final_score:.4f}")